[材料建模中的神经网络潜力](https://link.springer.com/referenceworkentry/10.1007/978-3-319-42913-7_56-1)

**摘要**:  
可靠的原子间电势的可用性对于进行复杂材料的计算机模拟是必要的.尽管像密度泛函理论这样的电子结构方法已在许多系统上获得了巨大成功,但这些方法的高计算成本严重限制了可以研究的科学问题.因此,近年来,在开发能够进行大规模仿真的更有效的潜力上花费了大量的精力.尤其是,机器学习潜力已引起人们的极大关注,因为它们有望将第一原理方法的准确性与力场的效率相结合.在这一章中,将回顾和讨论一类重要的采用人工神经网络的机器学习潜力.

# 引言

人工神经网络(NNs)的功能非常强大,目前广泛用于机器学习(ML)应用程序中(Bishop 1996 ; Haykin 2011):语音和手写识别,自动驾驶车辆,商业智能,工业流程控制和游戏,仅举几例.神经网络能够处理ML算法针对的两种主要类型的问题:分类和回归.本章重点介绍如何将NN用于特定的回归问题,即**对势能面(PES)的预测,即势能作为系统中原子位置的函数**.这种神经网络被称为神经网络势(NNP::neural network potentials)(Handley and Popelier 2010; Behler 2011b,2017).

PES是材料建模,理论凝聚态物理和计算化学中许多问题的核心.它给出了不同原子构型的相对稳定性,并直接与例如机械性能,缺陷分布,反应速率,热力学平衡,光谱特征以及许多其他性能相关.

有多种计算PES的方法,其中材料建模社区中最常用的方法包括电子结构方法,最突出的密度泛函理论(DFT)(Parr和Yang 1989)),以及各种原子或什至粗粒度的电势.DFT计算已被证明对许多实验性质都具有相当好的预测能力.但是,如果对大型系统进行建模,则DFT计算对计算的要求很高.这通常将DFT的适用性限制在数百个原子的静态(单点)计算或数百皮秒量级的短动态模拟中.另一方面,原子势更近似,因此以比DFT低得多的计算成本为PES提供估计.因此,它们实际上可以应用于大规模仿真.所需参数通常适合于重现建模系统的某些实验性质,或重现一些关键结果,例如DFT计算得出的能量.后一种方法是所谓的多尺度建模的示例,其中使用来自较高理论水平(DFT)的信息来参数化较低理论水平.

神经网络的潜力在许多方面类似于力场.但是,与力场不同,NNP的功能形式并非基于任何物理近似.取而代之的是,利用了神经网络的极大灵活性,并且对NNP进行了参数化,以再现从DFT参考计算或从其他电子结构方法获得的PES.然后,NNP提供了一种计算上便宜的方法来预测PES,这使得可以在大规模蒙特卡洛法和分子动力学模拟中使用NNP来采样构型和相空间.在NNP参数化过程中,NN"学习"不同分子和结构基序的稳定性.因此,NNP本质上是"反应性的,"的意思是,只要提供相关的训练数据,就可以准确地描述化学反应或重大的结构重排,包括共价键的断裂和形成.对于最终用户,使用神经网络电位或反作用力场之间没有明显的区别.两种类型的方法都以相似的计算成本提供PES,并且可以应用于相似类型的反应性系统.

神经网络已经使用了超过二十年的势能表示表面开始多伦和同事(Blank等人的开拓性工作1995年).第一代NNP受限于仅包含几个原子的小分子系统的限制,但在2007年可以通过引入高维NNP来克服此限制(Behler和Parrinello 2007).近年来,除了神经网络以外,机器学习方法也以类似的方式用于构建复杂系统原子模拟的潜力.例如,高斯近似势(Bartók等,2010),核岭回归(Rupp等,2012)等方法)和支持向量机(Balabin和Lomakina 2011)也可以用来描述PES和相关数量.Behler(2016)综述了基于机器学习的原子间电势的最新进展.

可以在许多教科书(例如Bishop 1996; Haykin 2011)中找到关于人工神经网络的整个理论的介绍,超出了本章的范围.因此,我们将专注于通常用于估计给定材料的PES的设计选择.势能是一个实数值,表示原子结构的输入特征是实数值,因此NNP是一个函数: $\chi: R^n→R $.

我们将讨论如何使用NNP来计算势能,如何进行从一组原子位置到合适的NN输入的转换,如何对NNP进行参数化,以及如何对NNP进行验证,以及其中的一些内容. NNP的优点和缺点.

# 前馈神经网络

图1显示了一个小的完全连接的前馈NN的示意图,该前馈NN定义了一个函数 $\chi: R^n→R $. 该 函数转换输入向量 $G = {(G_1,G_2)}^T$ 到输出值 E.该函数在权重矩阵 $a^{(0)},b^{(0)},a^{(1)},b^{(1)},a^{(2)},b^{(2)})$ 中包含几个参数, 定义  $ E = \chi(G_1,G_2; a^{(0)},b^{(0)},a^{(1)},b^{(1)},a^{(2)},b^{(2)})$. 示例NN由一个包含两个节点的输入层,两个分别包含三个节点的隐藏层以及一个包含一个节点的输出层组成. 此外,固定值1的偏置节点连接到隐藏层和输出层(以蓝色显示)中的所有节点. 示例NN的架构为2-3-3-1.层0中的输入 $(G_1,G_2)$ 对应于值$(y^{(0)}_1,y^{(0)}_2$.

![fig1](https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-42913-7_56-1/MediaObjects/116042_0_En_56-1_Fig1_HTML.png)

Fig1: 2-3-3-1前馈神经网络及激活函数 $f^{(k)}(x)$ 的典型选择. 
NN定义了函数: $ E = \chi(G_1,G_2; a^{(0)},b^{(0)},a^{(1)},b^{(1)},a^{(2)},b^{(2)})$.  
$y_i^{(k)}$ 和 $x_j^{(k)}$ 之间的空心箭头表示在权重矩阵 $a^{(k)}$ 中的元素 $a_{ij} ^{(k)}$; 明确示出了 $a^{(0)}$ 的几个元素.
类似地,蓝色箭头表示偏差权重. 实心箭头表示激活函数 $f^{(k)}$ 的应用.

计算从左到右进行:从输入层值开始,计算第一隐藏层值 $y^{(1)}  = {(y_1^{(1)},y_2^{(1)},y_3^{(1)})}^T$, 并生成一个偏置节点被添加; 然后将这些值用于计算第二隐藏层中的 $y^{(2)}  = {(y_1^{(2)},y_2^{(2)},y_3^{(2)})}^T$ 并添加一个偏置节点; 这些值被用于计算最终的输出值 E.

上一层 $y^{(k -1)}$ 中的值用于通过中间向量 $x^{(k)}$ 计算下一层中$y^{(k)}$的值:

$\displaystyle \begin{aligned} {\mathbf{x}}{{}^{(k)}}= ({\mathbf{a}}{{}^{(k-1)}})^{\mathrm{T}} {\mathbf{y}}{{}^{(k-1)}} +({\mathbf{b}}{{}^{(k-1)}})^{\mathrm{T}}. \end{aligned} \tag{1}$

在等式中,如图1所示,向量 $x^{(k)}$ 的每个元素都是按 $({\mathbf{b}}{{}^{(k-1)}})^{\mathrm{T}}$ 中的偏置权重和 ${\mathbf{y}}{{}^{(k-1)}}$ 元素的线性组合的总和计算的,系数为权重矩阵 ${\mathbf{a}}{{}^{(k-1)}}$. 例如,在图1中, $x_1^{(1)}  =  a 11 (0)y 1 (0)  +  a 21 (0)y 2 (0)  + b 11 (0)$. a (k -1)和b (k -1)的元素是在将NN用于能量预测之前必须确定的参数(NN权重);稍后显示如何确定.偏置节点的目的(具有1的值,在图以蓝色显示.1在层)ķ  - 1是添加不依赖于值的一些所需的常量ý (ķ -1) ,对中的每个元素x (k)(即到下一层).向量x (k)的值然后通过激活函数(有时称为传递函数或基函数)进行变换f (k)(x):

$\displaystyle \begin{aligned} y _{i}{{}^{(k)}} = f{{}^{(k)}}(x_i{{}^{(k)}}) \end{aligned} \tag{2}$

对于隐藏层,通常使用非线性S型激活函数,例如,逻辑函数或双曲正切f(x)= tanh(x).隐藏层激活功能还有其他可能的选择.对于输出层,线性激活函数f(x)=  x用于避免NN输出值的约束范围. F(x )= 11个+ Ë X p(- X )

为了使NN预测系统的势能,输入向量G必须包含有关确定势能的系统的所有信息.在没有任何外部场的情况下,势能在旋转和平移下是不变的.因此,**一个糟糕的选择是简单地选择原子的笛卡尔坐标作为输入向量,因为它们不具有这些不变性.更好的选择是内部坐标,例如原子间距离和键角,这些坐标在设计上在旋转和平移时不变**.只要内部坐标的数量是可管理的(例如描述小分子的PES),此方法就很好用.但是,对于包含数千个或更多自由度的大型系统,此方法很快变得棘手,因为输入要素的数量变得非常大.

代替让NN描述整个系统的PES("低维" NNP),可以构造描述单个原子的PES的NN.通过将几个这样的原子NN组合在一起,可以限制输入特征的数量,同时保留对具有数千个原子的大型系统进行建模的能力.这些高维NNP将成为本章其余部分的重点.高维NNP通常与所谓的对称函数一起用作输入特征.下一节将对它们进行描述.

# 高维神经网络电位和对称函数
在高维神经网络电势中,总能量计算为"原子"能量E i的总和:

$\displaystyle \begin{aligned} E = \sum_{i=1}^{N^{\mathrm{atom}}} E_i \end{aligned} \tag{3}$

其中N 原子是系统中的原子数.每个原子能E i都取决于元素相关的前馈NN,其中输入特征描述了原子周围的化学环境.这类似于如何使用原子环境上的总和(例如,键长度和角度)来计算许多力场中的总能量.然而,高维神经网络具有相当灵活的功能形式,可以轻松捕获高阶多体效应.实际上,通常使用称为对称函数的特定多体函数来计算输入特征.
对于具有一个以上元素的多组件系统,每个元素采用一个NN.例如,为了描述CuAgAu合金,将创建三个元素NN,一个用于Cu,一个用于Ag,一个用于Au,并且每个NN对相应元素的每个原子调用一次.可以将不同的NN体系结构(输入要素的数量,隐藏层的数量和/或每个隐藏层的节点数量)用于不同的元素.系统中所有Cu原子的原子能将使用相同的Cu特定NN进行评估,只有输入向量的数值不同,因为不同原子周围的原子环境可能会发生变化.这种处理确保了总能量相对于输入文件中提供Cu原子的顺序在排列上不变.图2示意性显示了如何使用高维神经网络电势计算总能量E.

![fig2](https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-42913-7_56-1/MediaObjects/116042_0_En_56-1_Fig2_HTML.png)
Fig2: 使用高维神经网络电势,对具有100个原子和两个元素I和J的系统的总能量E的评估的示意图. 在该示例中,Nsym(I)=40,Nsym(J)=30. 每个原子能 Ei 的评估如图1所示.

下面,我们指的是使用的索引特定原子我,Ĵ,和ķ以及使用大写字母的元素我,Ĵ,和ķ,并且我们使用符号我  ∈  予以指定原子我是元件的予.所有原子位置以及相应元素的集合表示为{ R,Z },其中{ R }表示笛卡尔坐标,Z表示元素.因此,{ R,Z}包含通常包含在蒙特卡洛模拟的单个帧或分子动力学模拟的单个帧(没有粒子速度)中的所有信息.

对于一个原子我  ∈  我,原子能ê 我等式 3计算为

$\displaystyle \begin{aligned} E_{i \in I} = \chi^I({\mathbf{G}}^I(i)) \end{aligned} \tag{4}$

其中χ 我表示NN为元件我.输入向量G I(i)是对称函数值的向量:

$\displaystyle \begin{aligned} {\mathbf{G}}^I(i \in I) = \begin{pmatrix} G_1^I(i, \{\mathbf{R}, Z\}) \\[0.1cm] G_2^I(i, \{\mathbf{R}, Z\}) \\[0.1cm] \vdots \\[0.1cm] G_{N^{\mathrm{sym}}(I)}^I(i, \{\mathbf{R}, Z\}) \end{pmatrix} \end{aligned} \tag{5}$

对称函数是原子周围化学环境的描述子.通常,仅考虑半径为R cut的截止球内的局部环境.这是通过渐缩函数f cut(R)实现的,该函数的值和斜率在R  =  R cut时平稳地衰减到0 .f cut(R)的常见选择是
FÇ ü Ť(R )= { tanh3( 1 − R[RÇ ü Ť)0[R ≤ [RÇ ü ŤR > RÇ ü Ť

$\displaystyle \begin{aligned} {f_{\mathrm{cut}}}(R) = \begin{cases} \tanh ^3 \left( 1 - \frac{R}{{R_{\mathrm{cut}}}} \right) & R \leq {R_{\mathrm{cut}}} \\ 0 & R > {R_{\mathrm{cut}}} \end{cases} \end{aligned} \tag{6}$

尽管其他形式的f 切也是可能的.用于原子"径向对称"的功能的一个例子我  ∈  我是Behler(2011A)

$\displaystyle \begin{aligned} G^{I}(i \in I, \{\mathbf{R}, Z\}; \,\, \eta, {R_{\mathrm{shift}}}, J) = \varphi\left( \sum_{\substack{j \in J \\ j \neq i}} e^{-\eta (R_{ij} - {R_{\mathrm{shift}}})^2} \cdot {f_{\mathrm{cut}}}(R_{ij}) \right). \end{aligned} \tag{7}$

此函数是如何元素的原子的描述符Ĵ(即可以是相同或不同的我)周围的原子分布我  ∈  予.它是高斯的总和乘以截止函数f cut(R).可选的特征缩放功能φ修改了对称函数G I输出的值的范围,这在NN参数化期间很有用.定标函数将在Sect中进一步讨论.4.2.两个参数η和R shift确定高斯函数的宽度和中心.数字图3a示出了等式中的求和值.7对于不同的距离,为的几个选择的值 η和 - [R 移位.此处,截止距离设置为 R cut  = 6Å的典型值,并且具有Eq的截止功能.使用6.黑线( η  = 0Å -2)等效于纯截断函数 f cut( R).


![fig3](https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-42913-7_56-1/MediaObjects/116042_0_En_56-1_Fig3_HTML.png)

Fig3: (a)等式(7)中径向对称函数的和, 对η和 Rshift 取一些不同的值,截止函数 fcut(R) 来自于等式(6).
(b) 公式(8)中对称函数角部分的求和, 对 ζ 和 λ 取某些选定值.

通常,对于中心元素I和相邻元素J的每种组合,在Eq中都有几种类型的函数.具有不同的η和/或R shift值的图7用作等式中NN的输入特征.5.与仅使用单个对称函数可以实现的指纹特性相比,使用多个这样的对称函数可以提供更好的原子环境指纹.例如,如果使用图3中 η  = 0Å -2的单个函数,则原子i周围的两个邻居(都在距离R  = 3Å处)得出相同的值G I(i)= 0.197作为距离R  = 2Å的单个邻居.通过使用几个对称函数,目标是将关于原子i周围原子环境的所有相关信息"编码" 到输入向量G I(i)中,并提供原子环境的结构指纹作为NN的输入.

等式中的径向对称函数.图7仅是距离相关的或换言之是球对称的.因此,还需要将角度依赖性并入对称函数中.角度对称函数的常见选择是Behler(2011a)

$\displaystyle \begin{aligned} \begin{array}{rcl} {} &\displaystyle &\displaystyle G^{I}(i \in I, \{\mathbf{R}, Z\}; \,\, \eta, \zeta, \lambda, J, K) \\ &\displaystyle &\displaystyle \ \ = \varphi\left( 2^{1-\zeta} \sum_{\substack{j \in J, \,\, k \in K \\ j \neq i, \,\, k \neq i \\ k \neq j}} (1+\lambda \cos \theta_{jik})^\zeta \cdot e^{-\eta(R_{ij}^2+R_{ik}^2+R_{jk}^2)} \cdot {f_{\mathrm{cut}}}(R_{ij}) \cdot {f_{\mathrm{cut}}}(R_{ik}) \cdot {f_{\mathrm{cut}}}(R_{jk}) \right) \end{array} \end{aligned}$

其中原子间距离- R IJ,- R IK,和- R JK和角度θ JIK三个原子之间我  ∈  我,Ĵ  ∈  Ĵ,和ķ  ∈  ķ被用于计算对称函数的值,对于每个可能的唯一组合围绕中心原子i的邻居j和k.同样,元素J和K可以与I和φ相同或不同.是缩放功能.在等式中 8,ζ确定的量,角术语是大约0角的范围,并且λ呈现,值为+ 1或- 1.对称函数的角度部分被示于图3 B,几个不同ζ和λ的值.值得注意的是,角部是周期性的,周期为360 ∘和对称围绕0 ∘和±180 ∘.
通过使用截止半径,在截止处会截断诸如静电相互作用之类的远程相互作用.对于许多类型的系统而言,这可能是有问题的.本节介绍了一种使用高维NNP将远程交互包含到PES中的方法.5.

高维NNP也可以用于评估作用在原子上的分析力,这对于诸如分子动力学模拟等应用程序至关重要.相对于某些原子坐标α的力为

$\displaystyle \begin{aligned} F_{\alpha} = - \frac{\partial E}{\partial \alpha} = - \sum_{j=1}^{N^{\mathrm{atom}}} \frac{\partial E_j}{\partial \alpha} = - \sum_{J \in \{Z\}} \sum_{j \in J} \sum_{\mu=1}^{N^{\mathrm{sym}}(J)} \frac{\partial E_j}{\partial G^J_{\mu}(j)} \cdot \frac{\partial G^{J}_{\mu}(j)}{\partial \alpha} \end{aligned} \tag{8}$

其中在所有的化学元素的最外总和运行Ĵ在系统中,并且其中是μ为元素个对称函数Ĵ为原子评价Ĵ  ∈  Ĵ. GĴμ(j )

In [6]:
ggtrans('Procurement')

采购


# 高维NNP的构建
对于某些给定的化学系统,构建高维NNP的过程涉及
采购培训和验证数据

每个元素的对称函数的选择(包括截止半径R cut)

每个元素的网络架构选择

每个元素的权重矩阵a (k)和b (k)的拟合

此外,必须严格评估和检查所拟合的NNP,以检查未充分描述的势能表面区域.
在高维NNP的典型应用中,每个基本NN使用1-3个隐藏层和每个隐藏层10-40个节点.对称函数的数量很大程度上取决于系统中化学元素的数量.通常,对于元素I,每个可能的相邻元素J使用 5-10个径向对称函数(等式7),并且每个可能的相邻元素J和K组合使用5-10个角度对称函数(等式8).对于所有对称函数,通常将截止距离R cut设置在6至10Å的范围内.

## 采购培训数据
培训数据的获取对于成功应用高维神经网络潜力至关重要.神经网络只能与已对其进行训练的数据一样好.因此,选择对建模材料准确的参考计算方法很重要.通常,某种形式的密度泛函理论(DFT)被用作参考方法.然后,训练集由一组结构组成,这些结构具有使用DFT确定的能量.可选地,作用在原子上的DFT力(包含有关PES的宝贵局部信息)也可以用于训练NN.训练程序的目的是使神经网络尽可能接近地复制参考能量和力.这是通过迭代优化权重矩阵来实现的每个基本NN的(k)和b (k).

在典型的情况下,越来越多的训练数据被迭代地添加到训练集中,以获得更好的神经网络潜力.从训练数据的一些初始集合,数个高维神经网络电位χ 0 (0) ,χ 0 (1) ,等被训练(如在下一节中所描述的).通过将这些电势应用于例如分子动力学模拟,可以识别描述欠佳的结构.这可以通过几种方式来完成:
1.
通过监视出现在模拟中的不同结构的对称函数的值.如果对称函数值不在训练集中出现的值的范围之内,或者如果对称函数的值仅很少出现在训练集中,则神经网络所做的预测可能不准确.

 
2.
通过比较能量和由不同的神经网络预测的力χ 0 (0) ,χ 0 (1)等上的相同结构.如果对于神经网络的不同拟合,能量和/或力有很大不同,则配置空间的各个部分将无法充分采样.

 
在上述任何方法识别的结构,然后使用参考方法重新计算并添加到所述训练集,随后是新的NN优化,从而产生新的NN电位χ 1 (0) ,χ 1 (1) ,等等.在该通过这种方式,将越来越多的训练数据迭代地添加到训练集中,直到认为NN能够准确描述模拟中出现的所有结构的PES.

## 对称函数的选择
对称函数的向量,等式中的G I.如图5所示,对于每个元素I,是什么使得NNP能够区分不同的结构.因此,一个重要的考虑,在设计为NNP一些化学系统时,是对称函数,其到评估当使用ģ 予.如果仅使用一些对称函数,则NNP可能无法可靠地区分不同的结构. G一世μ

实际上,通常根据经验选择和完善对称函数集,直到NNP给出令人满意的结果为止.一组"好的"对称函数通常包含满足以下条件的函数:
对于训练集中的所有原子,给定对称函数的值并不相同.更一般而言,当针对训练集中的不同原子计算某个对称函数的值的范围时,不应太小,因为NN可以将总能量的大变化分配给对称函数值的小变化.此外,应分析某个对称函数的值的分布,以确保值的范围不受少数异常值的支配.

对称函数集涵盖了一系列化学上有意义的距离.例如,衰减最快的径向对称函数(具有最大η的等式7)应在元素I和J的原子之间的尽可能短的有意义键的距离附近衰减.

经受非常大的力(通过参考方法计算)的原子必然存在于实质上不同的环境中,因此对于至少一种对称函数而言,它们应具有实质上不同的值.如果不是这种情况,则需要增加对称函数集.

任何两个对称函数之间的相关性不应太大.

将输入特征"预处理"到神经网络可能很有用.等式中示例对称函数中的可选缩放函数φ.例如,图7和图8可以确保训练集中不同结构的对称函数值都在某个预定范围内,例如[-1,1].的定义φ μ会话,对于对称函数G ^ μ,是

$\displaystyle \begin{aligned} \varphi _\mu(x) = \frac{ 2(x - G_\mu^{\circ, \mathrm{min}}) }{G_\mu^{\circ, \mathrm{max}} - G_\mu^{\circ, \mathrm{min}}} - 1 \end{aligned} \tag{9}$

其中和是在训练集中为相应的非标度对称函数获得的最小值和最大值.这种特征缩放有助于平衡不同对称函数的相对重要性.也可以使用其他类型的特征缩放功能. G∘ ,中号我Ñμ G∘ ,中号一个Xμ G∘μ

## 神经网络权重的优化
每个化学元素的NN权重参数a (k)和b (k)的优化都从对其值的初始猜测开始.初始权重可以简单地是随机数,但是也存在几种可以选择初始权重的方法,以最大程度地减少训练神经网络所需的时间,例如Nguyen和Widrow(1990).设置连接最后一个隐藏层和输出层的权重也可能很有用,以使初始预测能量的平均值和标准偏差(在任何训练之前)与训练集中参考能量的平均值和标准偏差相匹配.参考能量总是提供一些单位.权重可以看作是无量纲的数字,以正确的单位给出NN能量.

权重的优化是通过迭代最小化成本函数Γ来实现的.优化迭代通常称为时期.成本函数通常被视为NN计算值与参考值之间平方差的加权平均值.例如,成本函数的能量,Γ ē,是

$\displaystyle \begin{aligned} \varGamma_E = \frac{1}{N^{\mathrm{struct}}} \sum_{i=1}^{N^{\mathrm{struct}}} w_{i,E} \left( \frac{E^{\mathrm{NN}}_i - E^{\mathrm{Ref}}_i}{N_i^{\mathrm{atom}}} \right)^2 \end{aligned} \tag{10}$

其中N struct是训练集中的结构数,w i,E是与结构有关的权重参数,是第i个结构中的原子数.通过为训练集中的不同结构选择不同的w i,E值,可以为训练集中的某些结构分配比其他结构更大的重要性.注意,与神经网络权重a不同,权重w i,E表示用户在训练集中结构i的重要性,由用户设置而不是拟合. ñ一个吨Ô 米一世(k)和b (k).实际上,通常给 训练集中的所有结构分配相同的权重w i,E = 1.
同样,成本函数的力量,Γ ˚F,成为

$\displaystyle \begin{aligned} \varGamma_F = \frac{1}{N^{\mathrm{struct}}} \sum_{i=1}^{N^{\mathrm{struct}}} \left[ \frac{w_{i,F}}{3N^{\mathrm{atom}}_i} \sum_{j=1}^{3N^{\mathrm{atom}}_i} (F^{\mathrm{NN}}_j - F^{\mathrm{Ref}}_j)^2 \right] \end{aligned} \tag{11}$

其中,内部和是训练集结构i中所有力分量的总和.如果NN优化为在能量和力两者,所以能够在之间的任一方交替Γ Ë和Γ ˚F成本函数对于每个优化步骤(其中,NN的权重一(ķ)和b (ķ)被更新),或将它们合并为一个成本函数 3 N一个吨Ô 米一世

$\displaystyle \begin{aligned} \varGamma = \varGamma_E + \varGamma_F \end{aligned} \tag{12}$ 

在这种情况下,与结构有关的权重w i,F通常设置为比w i,E小得多的数字,因为对于结构i,只有一个能量E i,但是力组件.成本函数的表达式也可以取决于优化算法.例如,某些算法在成本函数中包含历史相关项. 3 N一个吨Ô 米一世
存在许多可能的优化算法.最简单的,最陡的下降(或"反向传播"(Rumelhart等,1986))根据以下规则更新权重:

$\displaystyle \begin{aligned} a_{ij}{{}^{(k)}}(t+1) = a_{ij}{{}^{(k)}}(t) - \eta \cdot \frac{\partial \varGamma}{\partial a_{ij}{{}^{(k)}}(t)} \end{aligned} \tag{13}$ 

其中t是历元数,η是学习率.对每个基本NN执行此步骤.但是,其他权重优化算法也存在并且经常使用,例如Levenberg-Marquardt算法(Levenberg 1944; Marquardt 1963)和全局扩展卡尔曼滤波器(Haykin 2001).这些算法的描述超出了本章的范围.
通常,一组NN权重的质量由能量和力的均方根误差RMSE来表征:

$\displaystyle \begin{aligned} \mathrm{RMSE}(E)\mathrm{ per atom} = \sqrt{ \frac{1}{N^{\mathrm{struct}}} \sum_{i=1}^{N^{\mathrm{struct}}} \left( \frac{ E^{\mathrm{NN}}_i - E^{\mathrm{Ref}}_i}{N_i^{\mathrm{atom}}} \right)^2 } \end{aligned} \tag{14}$

$\displaystyle \begin{aligned} \mathrm{RMSE}(F) = \sqrt{ \frac{1}{N^{\mathrm{struct}}} \sum_{i=1}^{N^{\mathrm{struct}}} \left[ \frac{1}{3N^{\mathrm{atom}}_i} \sum_{j=1}^{3N^{\mathrm{atom}}_i} (F^{\mathrm{NN}}_j - F^{\mathrm{Ref}}_j)^2 \right] } \end{aligned} \tag{15}$

RMSE(E)通常被报告为"每个原子"的归一化值,因为与包含几个原子的结构相比,包含多个原子的结构通常在能量上具有更大的绝对误差.对于RMSE(F),将内部总和应用于训练集中第i个结构的所有力分量. 3 N一个吨Ô 米一世

良好的NN的RMSE值较小.文献中报道的典型值约为每个原子的RMSE(E)= 1 meV和RMSE(F)= 100 meV /Å.但是,什么才是合适的,取决于NN的应用方式.此外,RMSE仅提供了一个简单的方法来衡量受训NN在训练数据上的表现.如果训练数据变化很大,例如,关于系统的化学组成,则在训练集中所有结构上平均的RMSE不一定帮助科学家找到结构或化学组成不能很好描述的结构. NN.在这种情况下,最好将训练集分为几个不同的组,例如,根据组成,分别计算每个组的RMSE.此外,探索神经网络产生的能量和力中的误差分布也很有帮助.如果NN在某些特定结构集上的表现不佳,w E和w F在训练程序中(方程10和11)或修改对称函数集,以便更好地描述原子化学环境.

## 训练集,验证集,测试集,过度拟合和不足拟合
采购的参考数据通常分为训练集和测试集.训练集由用于确定NN权重参数的结构组成.测试集包含不用于训练NN的其他参考数据.通过使用测试集,可以通过监视两个集的成本函数,在尚未训练的NN的结构上评估其质量.

测试集的使用有助于检测所谓的过拟合(也称为高方差).图4显示了过度拟合的示例在简单的一维情况下:尽管由蓝线表示的NN很好地再现了每个训练点(红色圆圈)的参考值,但NN对位于训练集之外的值(橙色三角形)做出了非常不准确的预测.因此,尽管训练集的RMSE很小,但测试集的RMSE却更大.这表明NN在训练集中的数据点之间进行的插值不佳.通常,由于神经网络架构太大(例如,相对于隐藏层数或每个隐藏层的节点数而言),会导致过度拟合.可以通过减少神经网络的大小,在更早的时期停止NN权重优化来解决过度拟合问题,Γ.正则项对权重a ij(k)的较大值进行惩罚,并且许多类型的正则项是可能的(例如,所谓的L 1和L 2正则化).可以在例如Haykin(2011)中找到关于与神经网络相关的正则化的更深入讨论.

![fig4](https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-42913-7_56-1/MediaObjects/116042_0_En_56-1_Fig4_HTML.png)

Fig4: (a)过度拟合,(b)拟合和(c)良好拟合的示例. 
红色圆圈表示训练集中的数据点,橙色三角形代表测试集中的点.

过拟合的反面是欠拟合(也称为具有高偏压),在图中所示4湾 欠拟合的特征在于训练和测试集的均方根误差(RMSE)高,如果没有足够的时间来进行神经网络权重优化,通常会发生欠拟合.如果运行更多的时期不能解决欠拟合问题,则可以帮助神经网络功能形式更加灵活,例如,通过向隐藏层添加更多节点.

图4 1c示出一个"般配",其中NN预测和基准数据之间的误差小,以及大致相等的训练集和测试集两者.

如果为了解决例如过度拟合而评估了几种神经网络体系结构,则存在最终选择的神经网络体系结构在测试集上具有极小的误差的危险,因为测试集用于选择神经网络.网络体系结构.在这种情况下,通常将用于评估NN体系结构的"测试集"称为验证集,然后为另一个独立的测试集评估最终选择的NN的错误.

# 包含远程静电
第5节中描述的神经网络潜力.3依赖于对称函数值的向量作为输入,对于任何给定的原子,只有环境中的原子(在截止半径R cut的范围内)才对对称函数值起作用.因此,这是本节中描述的高维NNP类型的基本假设.3可以从局部原子环境中计算出系统的势能.某些类型的相互作用(尤其是静电相互作用)仅随着距离的增加而缓慢衰减,即使两个相互作用原子之间的距离大于R cut,也可能对能量和力产生相当大的影响.

静电贡献的计算需要有关电荷分布的信息.这可以通过例如使每个原子与点电荷缔合来实现.这些电荷的符号和大小可以固定在某些预定的值(取决于元素),也可以通过某种方式"即时"确定.已与NNP结合使用的后一种方法的一个示例是使用另一种不同的NNP(Artrith等人2011 ; Morawietz等人2012).然后,该第二NNP将适合于再生原子电荷,类似于先前引入的NNP类型如何用于确定原子能.使用这种方法,必须在训练集中提供一些原子电荷的近似值.通常,使用可以从电子结构计算中得出的多种原子电荷中的一种,例如Mulliken电荷(Mulliken 1955),Bader电荷(Bader 1985)或Hirshfeld电荷(Hirshfeld 1977).图5说明了这种方法,其中使用一个高维NNP来计算"短程"能量E short它仅取决于本地原子环境,并且使用第二个高维NNP来估计用于计算"远距离"能量E long的原子电荷(例如,通过应用库仑定律或Ewald加法).然后,获得的势能为E  =  E short  +  E long.用于不同元素以计算原子电荷的NN体系结构和对称函数不必与用于计算原子能的那些相同.

![fig5](https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-42913-7_56-1/MediaObjects/116042_0_En_56-1_Fig5_HTML.png)

Fig5: 使用与图2中相同的化学系统和颜色编码来说明两个高维神经网络电势.上部的NN(白色背景)产生短程能量Eshort; 下部的NN(灰色背景)产生用于计算Elong的原子电荷qi. 它们加在一起给出总能量E.

不管用于生成原子电荷和计算E long的方案如何,至关重要的是,应将短距离NN拟合为仅重现E tot和E long之间的差,以避免重复计算对E的贡献.总能量.在E long的计算中,已证明使用在短距离处屏蔽的库仑电势是有益的,因为否则NN可能需要为短程能量拟合更波纹的势能表面.

相对于某些原子坐标α的力变为: 

$\displaystyle \begin{aligned} F_\alpha = F_\alpha^{\mathrm{short}} + F_\alpha^{\mathrm{long}} = - \frac{\partial E^{\mathrm{short}}}{\partial \alpha} - \frac{\partial E^{\mathrm{long}}}{\partial \alpha} \end{aligned} \tag{16}$

其中的计算公式如下.8.对于非周期性系统,可以计算为: 

$\displaystyle \begin{aligned} \begin{array}{rcl} F_\alpha^{\mathrm{long}} &\displaystyle =&\displaystyle - \frac{1}{2} \frac{\partial}{\partial \alpha} \sum_{i=1}^{N^{\mathrm{atom}}} \sum_{\substack{j=1 \\ j \neq i}}^{N^{\mathrm{atom}}}\\ \frac{q_iq_j}{R_{ij}} &\displaystyle =&\displaystyle - \frac{1}{2} \sum_{i=1}^{N^{\mathrm{atom}}} \sum_{\substack{j=1 \\ j \neq i}}^{N^{\mathrm{atom}}} \frac{1}{R_{ij}^2} \left[ \frac{\partial q_i}{\partial \alpha} q_jR_{ij} + q_i\frac{\partial q_j}{\partial \alpha}R_{ij} - q_iq_j\frac{\partial R_{ij}}{\partial \alpha} \right] \end{array} \end{aligned} \tag{17}$ 

其中q i是原子i上的电荷.如果使用具有对称函数的NNP来确定原子电荷,如图5所示,则可以证明:

$\displaystyle \begin{aligned} F_{\alpha}^{\mathrm{long}} = \sum_{J \in \{Z\}} \sum_{j \in J} \sum_{\substack{i=1 \\ i \neq j}}^{N^{\mathrm{atom}}} \frac{q_i}{R_{ij}} \cdot \left[ \frac{1}{2} \frac{q_j}{R_{ij}} \frac{\partial R_{ij}}{\partial \alpha} - \sum_{\mu=1}^{N^{\mathrm{sym}}(J)} \frac{\partial q_j}{\partial G_\mu^J(j)} \frac{\partial G_\mu^J(j)}{\partial \alpha} \right] . \end{aligned} \tag{18}$

对于周期性系统,其中使用例如Ewald求和来评估静电能,尽管的表达式变得更加复杂,尽管它可以类似的方式导出. F升Ò Ñ 克α

# 神经网络潜力的应用和局限性

NNP,并且特别是高维 NNPs,已经开发并应用到一系列不同的分子和材料的(综述参见Behler 2014,2017年). 一些例子包括硅(Behler等人2008),碳(Khaliullin等人2011),钠(Eshet等人2012),氧化锌(Artrith等人2011),碲化锗(Sosso等人2012),铜(Artrith和Behler 2012),ZnO上的Cu簇(Artrith等人2013),Cu-Au纳米合金(Artrith和Kolpak 2015),铜上的水(Natarajan和Behler 2016),二氧化钛(Artrith and Urban2016年),金(Boes等人2016年),铜-钯-银合金(Hajinazar等人2017年),Ru上的N2(Shakouri等人2017年)和ZnO上的水(Quaranta等人2017年).

由于使用这种灵活的方法可以以较低的计算成本获得空前的准确性,因此NNP的开发以及NNP方法的发展,以及其他类似的 ML 潜在方法的开发,目前是一个非常活跃的研究领域.本章已经介绍了 高维NNP 的一些局限性和缺点,包括:
大型训练集的迭代构造.典型的训练集包括数以千计的结构,必须使用**参考电子结构方法**进行计算.此外,新训练集结构的生成通常以经验方式完成,例如使用高温分子动力学模拟.

化学系统中元素的数量有限.在系统中有超过三个或四个化学元素时,描述原子周围局部化学环境所需的对称函数的数量变得非常大.对于此类系统,另一种类型的输入功能可能会有所帮助.

复杂且耗时的装配过程.NNP包含许多需要拟合的权重.即使使用高级拟合算法,获得良好的拟合也通常是一个耗时的过程.如本节所述,如果包含远程静电,该问题会更加严重.

# 小结

近年来,神经网络势(NNP)已成为原子材料建模仿真的非常有用的工具.它们以较低的计算成本提供系统的势能面(PES: potential energy surface),而精度非常接近于第一原理方法.由于NNP具有灵活的功能形式,因此它们可用于在平等的基础上描述所有类型的原子相互作用(共价键,分散键,氢键等). 在重大的远距离静电相互作用的情况下,可以使用一些确定参考原子电荷或多极的方案扩展NNP,以包括对总能量的远距离贡献.
虽然 NNP 适合通过电子结构计算来复制参考数据(通常是总能量和原子力),由于本质上非物理的但纯粹的数学功能形式,因此需要仔细验证所获得的电势.
在高维NNP的情况下,系统中的每个化学元素都适合一个NN.然后,元素特定的 NN 围绕特定类型的原子描述了PES. 使用例如对称函数,将本地原子环境转换为 NN 的输入. 然后,通过对所有原子贡献求和来计算总能量.  
与参考电子结构方法相比,与NNP相关的误差可以非常小(每个原子<1 meV),这使NNP成为将来在材料建模和仿真中的应用的有前途的工具.

# 参考文献

Artrith N,Behler J(2012)金属表面的高维神经网络电势:铜的原型研究.Phys Rev B 85:045439
ADSCrossRefGoogle Scholar
Artrith N,Kolpak AM(2015)使用反应性ANN势在热平衡状态下对Cu-Au纳米合金进行热力学的经典力学模拟.Comput Mater Sci 110:20
CrossRefGoogle学术搜索
Artrith N,Urban A(2016)一种用于原子材料模拟的人工神经网络电位的实现:TiO 2的性能.Comput Mater Sci 114:135–150
CrossRefGoogle学术搜索
Artrith N,Morawietz T,Behler J(2011)多组分系统的高维神经网络潜力:氧化锌的应用.Phys Rev B 83:153101
ADSCrossRefGoogle Scholar
Artrith N,Hiller B,Behler J(2013)金属和氧化物的神经网络电势–首次应用于氧化锌铜簇.实体状态Solidi B 250:1191–1203
ADSCrossRefGoogle Scholar
Bader R(1985)分子中的原子.Acc Chem Res 18:9
CrossRefGoogle Scholar
Balabin RM,Lomakina EI(2011)支持向量机回归(LS-SVM)-人工神经网络(ANN)的替代方法,用于分析量子化学数据?Phys Chem Chem Phys 13:11710
CrossRefGoogle学术搜索
BartókAP,Payne MC,Kondor R,CsányiG(2010)高斯逼近势:没有电子的量子力学的准确性.Phys Rev Lett 104:136403
ADSCrossRefGoogle Scholar
Behler J(2011a)以原子为中心的对称函数,用于构建高维神经网络电位.J Chem 134:074106
ADSCrossRefGoogle Scholar
Behler J(2011b)化学中的神经网络势能面:用于大规模模拟的工具.Phys Chem Chem Phys 13:17930–17955
CrossRefGoogle学术搜索
Behler J(2014)用高维神经网络电势表示势能面.J Phys Condens Matter 26:183001
CrossRefGoogle学术搜索
Behler J(2016)观点:原子模拟的机器学习潜力.J Chem 145(17):170901
ADSCrossRefGoogle Scholar
Behler J(2017)用于大分子和凝聚态系统反应模拟的第一原理神经网络潜力.Angew Chem Int Ed 56:12828
CrossRefGoogle学术搜索
Behler J,Parrinello M(2007)高维势能面的广义神经网络表示.Phys Rev Lett 98:146401
ADSCrossRefGoogle Scholar
Behler J,MartoňákR,Donadio D,Parrinello M(2008)利用高维神经网络电位对硅高压相进行的元动力学模拟.Phys Rev Lett 100:185501
ADSCrossRefGoogle Scholar
Bishop CM(1996)用于模式识别的神经网络.牛津大学出版社,牛津
zbMATHGoogle Scholar
Blank TB,Brown SD,Calhoun AW,Doren DJ(1995)势能面的神经网络模型.J Chem Phys 103:4129–4137
ADSCrossRefGoogle学术搜索
Boes JR,Groenenboom MC,Keith JA,Kitchin JR(2016)神经网络和ReaxFF比较Au属性.Int J Quantum Chem 116:979–987
CrossRefGoogle Scholar
Eshet H,Khaliullin RZ,KühneTD,Behler J,Parrinello M(2012)高压下钠的异常熔化行为的微观成因.Phys Rev Lett 108:115701
ADSCrossRefGoogle Scholar
Hajinazar S,Shao J,Kolmogorov AN(2017)分层构建基于神经网络的多组分材料原子间模型.Phys Rev B 95:014114
ADSCrossRefGoogle Scholar
Handley CM,Popelier PLA(2010)用人工神经网络拟合的势能面.J物理化学A 114:3371–3383
CrossRefGoogle学术搜索
Haykin S(2001)卡尔曼滤波和神经网络.Wiley,Hoboken
CrossRefGoogle学术搜索
Haykin S(2011)神经网络和学习机.培生教育,新德里
Google学者
Hirshfeld FL(1977)用于描述分子电荷密度的键合原子片段.Theor Chim Acta 44:129–138
CrossRefGoogle Scholar
Khaliullin RZ,Eshet H,KühneTD,Behler J,Parrinello M(2011)直接石墨-金刚石相变的成核机理.Nat Mater 10:693–697
ADSCrossRefGoogle Scholar
Levenberg K(1944)一种用于求解最小二乘非线性问题的方法.Quart Appl Math 2:164–168
MathSciNetCrossRefGoogle Scholar
Marquardt DW(1963)一种用于非线性参数的最小二乘估计的算法.SIAM J Appl Math 11:431–441
MathSciNetCrossRefGoogle Scholar
Morawietz T,Sharma V,Behler J(2012)基于与环境有关的原子能和电荷的水二聚体的神经网络势能面.J Chem 136:064103
ADSCrossRefGoogle Scholar
Mulliken RS(1955)关于LCAO-MO分子波函数的电子种群分析.I.J Chem Phys 23:1833
ADSCrossRefGoogle Scholar
Natarajan SK,Behler J(2016)固液界面的神经网络分子动力学模拟:低指数铜表面的水.Phys Chem Chem Phys 18:28704
CrossRefGoogle学术搜索
Nguyen DH,Widrow B(1990)用于自学习控制系统的神经网络.IEEE Con​​trol Syst Mag 3:18–23
CrossRefGoogle Scholar
Parr RG,Yang W(1989)原子和分子的密度泛函理论.牛津大学出版社,牛津
谷歌学者
Quaranta V,HellströmM,Behler J(2017)水-ZnO界面上的质子转移机制:预溶剂化的作用.J物理化学通讯8:1476
CrossRefGoogle学术搜索
Rumelhart DE,Hinton GE,Williams RJ(1986)通过反向传播错误学习表示.自然323:533–536
ADSCrossRefGoogle Scholar
Rupp M,Tkatchenko A,MüllerKR,von Lilienfeld OA(2012)通过机器学习对分子雾化能量进行快速而准确的建模.Phys Rev Lett 108:058301
ADSCrossRefGoogle Scholar
Shakouri K,Behler J,Meyer J,Kroes GJ(2017)活性气体表面动力学中表面声子的精确神经网络描述:N 2 + Ru(0001).J物理化学通讯8:2131
CrossRefGoogle学术搜索
Sosso GC,Miceli G,Caravati S,Behler J,Bernasconi M(2012)相变材料GeTe的神经网络原子势.Phys Rev B 85:174103
ADSCrossRefGoogle Scholar